In [1]:
!pip install trtokenizer
!pip install TurkishStemmer
!pip install https://huggingface.co/turkish-nlp-suite/tr_core_news_trf/resolve/main/tr_core_news_trf-any-py3-none-any.whl


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.2/849.2 MB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 883.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 919.6/919.6 kB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 107.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 87.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 35.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using

In [2]:
import pandas as pd
import re
import string
from trtokenizer.tr_tokenizer import SentenceTokenizer
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
from snowballstemmer import stemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from gensim.models import Word2Vec
from TurkishStemmer import TurkishStemmer
import spacy
nlp = spacy.load("tr_core_news_trf")

# Importing as module.
import tr_core_news_trf
nlp = tr_core_news_trf.load()
import spacy
from spacy.lang.tr.stop_words import STOP_WORDS



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
df = pd.read_excel('tweetData.xlsx')

In [4]:
df = df.dropna()

In [5]:
df.shape

(1976, 2)

In [6]:
def remove_special_characters(text):
    try:
      text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
      return text
    except:
        pass

def to_lowercase(text):
    try:
      return text.lower()
    except:
        pass

def remove_extra_spaces(text):
    try:
      return  ' '.join(text.split())
    except:
        pass

def remove_html_tags(text):
    try:
      clean = re.compile('<.*?>')
      return re.sub(clean, '', text)
    except:
        pass

def remove_urls(text):
    try:
      return re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    except:
        pass

def remove_numbers(text):
    try:
      return re.sub(r'\d+', '', text)
    except:
        pass

def remove_punctuation(text):
    try:
      translator = str.maketrans('', '', string.punctuation)
      return text.translate(translator)
    except:
      pass

In [7]:
df['Tweets'] = df['Tweets'].apply(to_lowercase).apply(remove_extra_spaces).apply(remove_punctuation).apply(remove_html_tags).apply(remove_urls).apply(remove_numbers)

In [8]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
df.Tweets.loc[15595:15599]

15595    kimmiş lan rahatsız olacak söylesin infaz edel...
15596                              i̇nstasina infaz edelim
15597    üstad latin abd ülkelerinde denediler direk ka...
15598    bu iki oevladının açık adreslerini verin infaz...
15599     kıbrısın tamamı alınmalı   ada diye bize yutt...
Name: Tweets, dtype: object

In [9]:
df.head()

,Tweets,Etiket
0,aynı bayrak aynı göğün altındayız biriz berabe...,1.0
1,çok istekli olan bu erkekler lütfen erkek çoc...,1.0
2,kurtlerle komşumuz alrabayiz beraberiz siz yal...,2.0
3,müslümanın seni beni ırkı olmaz dini olur kims...,1.0
4,soramazsına hoşgeldiniz bu gün bilge ile berab...,2.0


In [10]:


# Cümle ve kelime ayrıştırıcı işlevler
def trtokenizer_sentence(text):
    try:
        return sent_tokenize(text, language='turkish')
    except Exception as e:
        print(f"Sentence tokenization error: {e}")
        return []

def trtokenizer_word(text):
    try:
        return word_tokenize(text, language='turkish')
    except Exception as e:
        print(f"Word tokenization error: {e}")
        return []


def remove_stopwords_spacy(text):
    turkish_stopwords = STOP_WORDS
    cleaned = [word for word in text if word not in turkish_stopwords]
    return cleaned


def spacy_lemme(text):
    # Join the list of words into a single string
    text = ' '.join(text)
    doc = nlp(text)
    lemme = []
    for t in doc:
      lemme.append(t.lemma_)
    return ' '.join(lemme)




# Cümle ve kelime ayrıştırma işlemlerini DataFrame'e uygulama
df['sentences'] = df['Tweets'].apply(trtokenizer_sentence)

# Kelimeleri çıkarmak için her cümleyi kelimelere böler ve tüm kelimeleri düz bir liste olarak toplar
df['words'] = df['sentences'].apply(lambda sentences: [word for sentence in sentences for word in trtokenizer_word(sentence)])

# Stop kelimeleri temizleme işlemini ve birleştirme işlemini DataFrame'e uygulama
df['cleaned_words'] = df['words'].apply(remove_stopwords_spacy)
df['lemmatized_words'] = df['cleaned_words'].apply(spacy_lemme)

# Sonuçları yazdırma
print(df.head())




                                              Tweets  Etiket  \
0  aynı bayrak aynı göğün altındayız biriz berabe...     1.0   
1  çok istekli olan bu erkekler  lütfen erkek çoc...     1.0   
2  kurtlerle komşumuz alrabayiz beraberiz siz yal...     2.0   
3  müslümanın seni beni ırkı olmaz dini olur kims...     1.0   
4  soramazsına hoşgeldiniz bu gün bilge ile berab...     2.0   

                                           sentences  \
0  [aynı bayrak aynı göğün altındayız biriz berab...   
1  [çok istekli olan bu erkekler  lütfen erkek ço...   
2  [kurtlerle komşumuz alrabayiz beraberiz siz ya...   
3  [müslümanın seni beni ırkı olmaz dini olur kim...   
4  [soramazsına hoşgeldiniz bu gün bilge ile bera...   

                                               words  \
0  [aynı, bayrak, aynı, göğün, altındayız, biriz,...   
1  [çok, istekli, olan, bu, erkekler, lütfen, erk...   
2  [kurtlerle, komşumuz, alrabayiz, beraberiz, si...   
3  [müslümanın, seni, beni, ırkı, olmaz, dini, ol...  

In [11]:
# Kelimeleri tokenize et
tokenize_metni = [metin.lower().split() for metin in df['Tweets']]

# Word2Vec modelini eğit
word2vec_model = Word2Vec(sentences=tokenize_metni, vector_size=100, window=5, min_count=1, workers=4)

# Her metin için ortalama kelime vektörlerini hesapla
def ortalama_vektor(metin):
    kelimeler = metin.lower().split()
    vektörler = [word2vec_model.wv[kelime] for kelime in kelimeler if kelime in word2vec_model.wv]
    return sum(vektörler) / len(vektörler) if vektörler else [0]*100

df['word2vec_vektör'] = df['Tweets'].apply(ortalama_vektor)

print(df['word2vec_vektör'])

0        [-0.0012957937, 0.002791498, 0.0027088244, -0....
1        [-0.0046601337, 0.0080273915, -0.00048469237, ...
2        [-0.0015658042, 0.005380033, -0.000978244, -0....
3        [-0.0052566207, 0.008143496, 0.00086617115, 0....
4        [-0.0052863876, 0.006084495, 0.004094764, 0.00...
5        [-0.0016295938, 0.003451394, -0.0006514524, -2...
6        [-0.0044722827, 0.010085605, -0.00025214546, 0...
7        [-0.0009769355, 0.0057813595, 0.0011610078, 9....
8        [-0.003224209, 0.011553913, -0.0022094953, 0.0...
9        [-0.0031060984, 0.0073678526, -0.0005052862, 0...
10       [-0.0033802849, 0.0069607142, -0.0005406245, 0...
11       [-0.005358598, 0.005318733, 0.0010739208, 0.00...
12       [-0.0062259254, 0.009533514, -0.00044922368, 0...
13       [-0.0076460023, 0.0062573883, 0.0025173156, 0....
14       [-0.0019983056, 0.006483823, 0.0008239681, -0....
15       [-0.0033109703, 0.0051504257, 0.0010100943, 0....
16       [-0.003621141, 0.007991859, -0.00068066263, 0..